In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTfusion.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTtexts_test.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTtokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3101
Total X variables: 182256

Text train example1


'20023kt 9999 prec0n CL0 CM0 15 08 q1021 22014G25KT 160V290 CAVOK 19/03 Q1021 NOSIG'


Sequences example 1


[778, 1, 2, 5, 3, 16, 12, 28, 1713, 710, 15, 43, 39, 28, 4]


Text train example2


...

[159, 1, 2, 5, 3, 48, 465, 128, 44, 15, 331, 380, 100, 4]


X


array([[   0,    0,    0,    0,    0,  778,    1,    2,    5,    3,   16,
          12,   28],
       [   0,    0,    0,    0,  778,    1,    2,    5,    3,   16,   12,
          28, 1713],
       [   0,    0,    0,  778,    1,    2,    5,    3,   16,   12,   28,
        1713,  710],
       [   0,    0,  778,    1,    2,    5,    3,   16,   12,   28, 1713,
         710,   15],
       [   0,  778,    1,    2,    5,    3,   16,   12,   28, 1713,  710,
          15,   43],
       [ 778,    1,    2,    5,    3,   16,   12,   28, 1713,  710,   15,
          43,   39],
       [   1,    2,    5,    3,   16,   12,   28, 1713,  710,   15,   43,
          39,   28],
       [   0,    0,    0,    0,    0,  159,    1,    2,    5,    3,   48,
         465,  128],
       [   0,    0,    0,    0,  159,    1,    2,    5,    3,   48,  465,
         128,   44],
       [   0,    0,    0,  159,    1,    2,    5,    3,   48,  465,  128,
          44,   15],
       [   0,    0,  159,    1,    2,    5,    3, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 '11': 6,
 '12': 7,
 '10': 8,
 '09': 9,
 '13': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTmodel.keras")


Epoch 1/50
356/356 [==============================] - 28s 69ms/step - loss: 3.7650 - accuracy: 0.2191
Epoch 2/50
356/356 [==============================] - 24s 68ms/step - loss: 2.6732 - accuracy: 0.3152
Epoch 3/50
356/356 [==============================] - 24s 68ms/step - loss: 2.4100 - accuracy: 0.3526
Epoch 4/50
356/356 [==============================] - 24s 68ms/step - loss: 2.2960 - accuracy: 0.3707
Epoch 5/50
356/356 [==============================] - 24s 68ms/step - loss: 2.2213 - accuracy: 0.3830
Epoch 6/50
356/356 [==============================] - 24s 69ms/step - loss: 2.1588 - accuracy: 0.3924
Epoch 7/50
356/356 [==============================] - 24s 68ms/step - loss: 2.1027 - accuracy: 0.4029
Epoch 8/50
356/356 [==============================] - 24s 68ms/step - loss: 2.0426 - accuracy: 0.4156
Epoch 9/50
356/356 [==============================] - 24s 68ms/step - loss: 1.9853 - accuracy: 0.4254
Epoch 10/50
356/356 [==============================] - 24s 69ms/step - loss: 1.931